In [ ]:
%%bash

pip install -q kaggle
mkdir -p ~/.kaggle
cp drive/MyDrive/kaggle.json ~/.kaggle/
chmod 600 /root/.kaggle/kaggle.json
kaggle competitions download -c how-are-you-feeling-fellas
unzip -qq how-are-you-feeling-fellas.zip

# How are you feeling Fellas (Kaggle Competition)

## Importing required modules

In [2]:
import pandas as pd
import numpy as np
import PIL
from tensorflow import keras
import tensorflow as tf
from matplotlib import pyplot as plt 
from sklearn.model_selection import train_test_split
import pathlib
from sklearn.preprocessing import LabelEncoder

In [3]:
df = pd.read_csv("train_set.csv")
df.head()

,id_img,paths,label
0,0,data\train\angry\0.jpg,angry
1,1,data\train\angry\1.jpg,angry
2,10,data\train\angry\10.jpg,angry
3,10002,data\train\angry\10002.jpg,angry
4,10016,data\train\angry\10016.jpg,angry


In [4]:
df["paths"] = df["paths"].str.replace("\\", "/")

<ipython-input-4-54a71692010b>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["paths"] = df["paths"].str.replace("\\", "/")


In [5]:
le = LabelEncoder()
df["label"] = le.fit_transform(df["label"])
print(le.classes_)

['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']


In [6]:
X = df["paths"]
y = df["label"]

In [7]:
def preprocess_image(X, y):
  X_preprocessed = []
  y_preprocessed = []
  for df_img in range(len(X)):
    try:
      image = tf.io.read_file(X[df_img])
      image = tf.image.decode_jpeg(image, channels=3)
      image = tf.image.resize(image, (64, 64))
      image /= 255.0
      X_preprocessed.append(image)
      y_preprocessed.append(y[df_img])
    except:
      filename = X[df_img]
      print("[-] Error in converting file : ", filename)
      pass
  
  return X_preprocessed, y_preprocessed

In [8]:
X.values[0]

'data/train/angry/0.jpg'

In [9]:
X, y = preprocess_image(X.values, y.values)

In [10]:
X = np.array(X)
y = np.array(y)

In [11]:
base_model = tf.keras.applications.VGG16(weights="imagenet", include_top=False, input_shape=(64, 64, 3))
base_model.trainable = False

In [12]:
inputs = tf.keras.layers.Input(shape=(64, 64, 3))
x = base_model(inputs)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(7, activation="sigmoid")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 7)                 3591      
                                                                 
Total params: 15,767,367
Trainable params: 1,052,679
Non-trai

In [14]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [15]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')

In [16]:
history = model.fit(X, y, epochs=120, callbacks=[early_stop])

Epoch 1/120
901/901 [==============================] - ETA: 0s - loss: 1.6499 - accuracy: 0.3519

901/901 [==============================] - 18s 16ms/step - loss: 1.6499 - accuracy: 0.3519
Epoch 2/120
899/901 [============================>.] - ETA: 0s - loss: 1.5490 - accuracy: 0.3966

901/901 [==============================] - 14s 15ms/step - loss: 1.5491 - accuracy: 0.3965
Epoch 3/120
901/901 [==============================] - ETA: 0s - loss: 1.5105 - accuracy: 0.4128

901/901 [==============================] - 14s 15ms/step - loss: 1.5105 - accuracy: 0.4128
Epoch 4/120
899/901 [============================>.] - ETA: 0s - loss: 1.4820 - accuracy: 0.4278

901/901 [==============================] - 14s 16ms/step - loss: 1.4819 - accuracy: 0.4278
Epoch 5/120
898/901 [============================>.] - ETA: 0s - loss: 1.4530 - accuracy: 0.4390

...

901/901 [==============================] - 14s 16ms/step - loss: 0.6787 - accuracy: 0.7165
Epoch 117/120
900/901 [============================>.] - ETA: 0s - loss: 0.6791 - accuracy: 0.7184

901/901 [==============================] - 14s 16ms/step - loss: 0.6792 - accuracy: 0.7184
Epoch 118/120
901/901 [==============================] - ETA: 0s - loss: 0.6699 - accuracy: 0.7245

901/901 [==============================] - 14s 16ms/step - loss: 0.6699 - accuracy: 0.7245
Epoch 119/120
900/901 [============================>.] - ETA: 0s - loss: 0.6695 - accuracy: 0.7248

901/901 [==============================] - 14s 16ms/step - loss: 0.6695 - accuracy: 0.7247
Epoch 120/120
898/901 [============================>.] - ETA: 0s - loss: 0.6605 - accuracy: 0.7302

901/901 [==============================] - 14s 16ms/step - loss: 0.6604 - accuracy: 0.7303


In [17]:
df_test = pd.read_csv("test_set.csv")
df_test.head()

,id_img
0,10052
1,10065
2,10079
3,10095
4,10121


In [18]:
def preprocess_image_test(X):
  X_preprocessed = []
  file_path = "data/test/"
  for df_img in range(len(X)):
    try:
      image = tf.io.read_file(file_path + str(X[df_img]) + ".jpg")
      image = tf.image.decode_jpeg(image, channels=3)
      image = tf.image.resize(image, (64, 64))
      image /= 255.0
      X_preprocessed.append(image)
    except:
      filename = X[df_img]
      print("[-] Error in converting file : ", filename)
      pass
  
  return X_preprocessed

In [19]:
X_test = preprocess_image_test(df_test["id_img"].values)

In [20]:
X_test = np.array(X_test)

In [21]:
predictions = model.predict(X_test)

221/221 [==============================] - 4s 16ms/step


In [22]:
classes = le.classes_

In [23]:
pred_tests = []
for pred in predictions:
  result_idx = np.max(pred)
  pred = pred.tolist()
  pred = pred.index(result_idx)
  pred = classes[pred]
  pred_tests.append(pred)

In [24]:
pred_tests

['fear',
 'sad',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'happy',
 'angry',
 'happy',
 'happy',
 'angry',
 'happy',
 'angry',
 'fear',
 'neutral',
 'angry',
 'sad',
 'angry',
 'happy',
 'happy',
 'angry',
 'angry',
 'angry',
 'neutral',
 'neutral',
 'happy',
 'neutral',
 'angry',
 'fear',
 'neutral',
 'angry',
 'angry',
 'angry',
 'happy',
 'angry',
 'sad',
 'happy',
 'fear',
 'sad',
 'angry',
 'angry',
 'fear',
 'neutral',
 'angry',
 'neutral',
 'happy',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'happy',
 'neutral',
 'angry',
 'neutral',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'sad',
 'angry',
 'happy',
 'neutral',
 'sad',
 'neutral',
 'neutral',
 'angry',
 'angry',
 'angry',
 'happy',
 'sad',
 'angry',
 'surprise',
 'sad',
 'angry',
 'fear',
 'sad',
 'angry',
 'sad',
 'happy',
 'neutral',
 'angry',
 'neutral',
 'fear',
 'angry',
 'fear',
 'angry',
 'neutral',
 'neutral',
 'angry',
 'sad',
 'sad',
 'happy',
 'angry',
 'fear'

In [25]:
df_samp = pd.read_csv("sample_submission.csv")
df_samp.head()

,id_img,label
0,10052,surprise
1,10065,surprise
2,10079,surprise
3,10095,surprise
4,10121,surprise


In [26]:
df_test["label"] = pred_tests

In [27]:
df_test.head()

,id_img,label
0,10052,fear
1,10065,sad
2,10079,angry
3,10095,angry
4,10121,angry


In [28]:
df_test.to_csv("pred_result.csv", index=False)